# Desenvolvimento da Carteira Consolidada

**Preparado por:** Vinicius Suguimoto

In [118]:
import os
import sys

module_path = os.path.abspath(os.path.join('..'))

if module_path not in sys.path:
    sys.path.append(module_path)

import aurum
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

In [31]:
# Teoricamente toda essa etapa de treinamento do modelo já estará encapsulada no
# modelo salvo na pasta models, contudo a partir desse diretório não consigo acessar
# a pasta. Treinaremos um modelo simples para consolidar os resultados

def treinar_modelo(TICKER):
    DATA = aurum.data.get_ohlcv(TICKER)
    DATA = aurum.ft.technical_indicators(DATA)

    TRAIN_SIZE = int(len(DATA) * .75)

    INDICADORES = [
        'RSI_14',
        'STOCHk_14_3_3',
        'STOCHd_14_3_3',
        'ROC_2',
        'ROC_5',
        'ROC_10',
        'SLOPE_3',
        'ATRr_5',
        'WILLR_14',
        'OBV',
        'OBV_ROC_14',
        'EMA_BUY_CROSS',
        'EMA_SELL_CROSS',
        'EMA_9_DISTANCE', 
        'EMA_21_DISTANCE',
        'BBAND_FechouFora_Lower',
        'BBAND_FechouFora_Upper'
    ]

    X_train = DATA.loc[TRAIN_SIZE:, INDICADORES]
    y_train = DATA.loc[TRAIN_SIZE:, 'Alvo']

    MODEL = aurum.model_training.create_setup(X_train, y_train)

    return MODEL

# Modelos para a Carteira

MODELO_PETR4 = treinar_modelo('PETR4.SA')
MODELO_CMIG3 = treinar_modelo('CMIG3.SA')
MODELO_ABEV3 = treinar_modelo('ABEV3.SA')

In [84]:
# Resultados consolidados

def model_results(TICKER, MODEL): 

    TEST = aurum.data.get_ohlcv(TICKER, TREINO=False)
    TEST = aurum.ft.technical_indicators(TEST)
    pred = MODEL.predict(TEST[MODEL.feature_names_in_])

    TEST['Ticker'] = TICKER
    TEST['Retorno do Modelo'] = pred * TEST['LEAK_Retorno']/4
    TEST['Retorno Buy and Hold'] = (TEST['Close'] - TEST.loc[0, 'Close'])/TEST.loc[0, 'Close']

    RETURN = TEST[['Ticker','Date', 'Retorno do Modelo','Retorno Buy and Hold']]

    return RETURN



In [85]:
RETORNO_PETR4 = model_results('PETR4.SA', MODELO_PETR4)
RETORNO_CMIG3 = model_results('CMIG3.SA', MODELO_CMIG3)
RETORNO_ABEV3 = model_results('ABEV3.SA', MODELO_ABEV3)

In [150]:
def wallet_return(RETURN_LIST, TICKER_WEIGHT):

    assert len(RETURN_LIST) == len(TICKER_WEIGHT)
    assert sum(TICKER_WEIGHT) == 1

    
    RETORNO_PONDERADO_MODELO = np.zeros(len(RETURN_LIST[0]))

    for i, retorno in enumerate(RETURN_LIST):

        RETORNO_PONDERADO_MODELO += retorno['Retorno do Modelo'] * TICKER_WEIGHT[i]

    RETORNO_ACUMULADO_MODELO = (1 + RETORNO_PONDERADO_MODELO).cumprod()

    RESULTADOS = pd.DataFrame(RETORNO_ACUMULADO_MODELO)
    RESULTADOS['Date'] = RETURN_LIST[0]['Date']
    RESULTADOS = RESULTADOS.rename({'Retorno do Modelo': 'Retorno da Carteira'}, axis=1)

    return RESULTADOS[['Date', 'Retorno da Carteira']]

In [151]:
RETORNO_GERAL = wallet_return([RETORNO_PETR4, RETORNO_CMIG3], [.5, .5])

In [157]:
import plotly.express as px

In [158]:
px.line(RETORNO_GERAL,x='Date', y='Retorno da Carteira')

In [152]:
RETORNO_GERAL


,Date,Retorno da Carteira
0,2022-06-30,1.003051
1,2022-07-01,1.005958
2,2022-07-04,1.003887
3,2022-07-05,1.004782
4,2022-07-06,1.012720
...,...,...
75,2022-10-17,1.251985
76,2022-10-18,1.251985
77,2022-10-19,1.239733
78,2022-10-20,1.224819
